# Stereo Disparity Map using Normalized Cross Correlation

This notebook computes a stereo disparity map for the images `image0110_c0.pgm` and `image0110_c1.pgm` using normalized cross correlation (NCC) as the similarity metric. The implementation follows the algorithm discussed in the lecture and does not use OpenCV's built-in stereo matching functions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cv2
import os

# Unzip images if not already extracted
notebook_dir = os.getcwd()
image_left_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteLeft', 'image0110_c0.pgm')
image_right_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteRight', 'image0110_c1.pgm')

img_left = cv2.imread(image_left_path, cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread(image_right_path, cv2.IMREAD_GRAYSCALE)

print("Images loaded:", img_left.shape, img_right.shape)

Images loaded: (481, 640) (481, 640)


## Stereo Matching with Normalized Cross Correlation

We implement the block matching algorithm using NCC as the similarity metric. The disparity is computed for each pixel by searching for the best match along the same row in the right image.

In [2]:
def compute_ncc_patch(patch1, patch2):
    """Compute normalized cross correlation between two patches."""
    mean1 = patch1 - np.mean(patch1)
    mean2 = patch2 - np.mean(patch2)
    numerator = np.sum(mean1 * mean2)
    denominator = np.sqrt(np.sum(mean1 ** 2) * np.sum(mean2 ** 2))
    if denominator == 0:
        return 0
    return numerator / denominator

def compute_disparity_ncc(img_left, img_right, max_disp=64, window_size=9):
    """Compute disparity map using NCC."""
    h, w = img_left.shape
    half_w = window_size // 2
    disparity_map = np.zeros((h, w), dtype=np.float32)
    for y in range(half_w, h - half_w):
        for x in range(half_w, w - half_w):
            best_ncc = -1
            best_disp = 0
            left_patch = img_left[y-half_w:y+half_w+1, x-half_w:x+half_w+1]
            for d in range(0, min(max_disp, x-half_w+1)):
                x_right = x - d
                right_patch = img_right[y-half_w:y+half_w+1, x_right-half_w:x_right+half_w+1]
                ncc = compute_ncc_patch(left_patch, right_patch)
                if ncc > best_ncc:
                    best_ncc = ncc
                    best_disp = d
            disparity_map[y, x] = best_disp
    return disparity_map

# Try different window sizes to find optimal one
#window_sizes = [5, 7, 9, 11, 13]
window_sizes = [5]
disparity_maps = []
for ws in window_sizes:
    print(f"Computing disparity map with window size {ws}...")
    disp = compute_disparity_ncc(img_left, img_right, max_disp=64, window_size=ws)
    disparity_maps.append(disp)

Computing disparity map with window size 5...


KeyboardInterrupt: 

## Visualize Disparity Maps

Display the computed disparity maps for different window sizes using the "jet" colormap with logarithmic scaling.

In [ ]:
fig, axs = plt.subplots(1, len(window_sizes), figsize=(20, 5))
for i, (disp, ws) in enumerate(zip(disparity_maps, window_sizes)):
    axs[i].imshow(disp, cmap='jet', norm=LogNorm(vmin=1, vmax=np.max(disp)+1))
    axs[i].set_title(f'Window size: {ws}')
    axs[i].axis('off')
plt.suptitle("Disparity Maps with Different Window Sizes (LogNorm, 'jet')")
plt.show()

## Final Disparity Map

The optimal window size can be chosen based on the visual quality of the disparity maps above. Below, we display the disparity map for the selected optimal window size.

In [ ]:
# Select optimal window size (e.g., 9)
optimal_idx = 1  # window_size=9
optimal_disp = disparity_maps[optimal_idx]

plt.figure(figsize=(10, 8))
plt.imshow(optimal_disp, cmap='jet', norm=LogNorm(vmin=1, vmax=np.max(optimal_disp)+1))
plt.title("Final Disparity Map (Window size = 9, LogNorm, 'jet')")
plt.axis('off')
plt.colorbar()
plt.show()